<a href="https://colab.research.google.com/github/MattiaVerticchio/PersonalProjects/blob/master/NewsClassification/ReutersNewsClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reuters Dataset



> **Abstract**
>
> 



## Framework setup
As central framework for this task we’ll use [Auto-Keras](https://arxiv.org/abs/1806.10282), an efficient neural architecture search system developed by DATA Lab at Texas A&M University. It leverages a variant of Bayesian Optimization to guide deep neural network morphism and find a good architecture for our task and dataset, using Keras and TensorFlow as backend.
First, we have to install it with its dependency, [Keras-Tuner](https://keras-team.github.io/keras-tuner/), which is the hyperparameter optimization library used by Auto-Keras.

In [1]:
%%bash
pip install -q git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1
pip install -q autokeras

## Imports
Now we can import the TensorFlow dataset loading tool and Auto-Keras, as well as NumPy to store images and labels in arrays.

In [2]:
from tensorflow.keras.datasets import reuters
import autokeras as ak
import numpy as np

## Loading the data
The dataset can be now loaded into four NumPy arrays. The data has an offset of 3 units, the first indeces, in fact are occupied as follows.

0. Padding
1. Start of sequence
2. Unknown word


In [3]:
offset = 3  # Index offset
(x_train, y_train), (x_test, y_test) =  reuters.load_data(num_words=1000, 
                                                          index_from=offset)
y_train = y_train.reshape(-1, 1)
y_test  = y_test.reshape(-1, 1)

2113536/2110848 [==============================] - 0s 0us/step


Let’s prepare the dictionary that maps indeces to words.


In [4]:
word_to_id = reuters.get_word_index()
word_to_id = {k: (v + offset) for k, v in word_to_id.items()}
word_to_id['PADDING'] = 0
word_to_id['START_OF_SEQUENCE'] = 1
word_to_id['UNKNKOWN'] = 2
id_to_word = {value: key for key, value in word_to_id.items()}

557056/550378 [==============================] - 0s 0us/step


Now we convert the lists of indeces to actual words using a dictionary map. 

In [5]:
x_train = list(map(lambda sentence: ' '.join(
    id_to_word[i] for i in sentence), x_train))
x_test = list(map(lambda sentence: ' '.join(
    id_to_word[i] for i in sentence), x_test))

x_train = np.array(x_train, dtype=np.str)
x_test  = np.array(x_test, dtype=np.str)

## Building the model
`TextClassifier` is the class responsible for model search. Here I set `max_trials=3` to avoid taking too much time for model exploration, but it can be set to any positive integer. It explores different model architectures by tree-based Bayesian Optimization search. We’ll try only the first one as it’s really time consuming on Google Colab’s GPU, where I’m running this notebook.

In [6]:
clf = ak.TextClassifier(              # Initialize the text classifier
    num_classes=None,                 # Infer the number of classes
    multi_label=False,                # Only one output
    loss='categorical_crossentropy',  # Select the loss metric
    metrics='accuracy',               # Metric to watch
    project_name="text_classifier",   # Name of the folder
    max_trials=3,                     # Just try three models
    directory=None,                   # Automatic folder creation
    objective="val_loss",             # Validation set crossentropy
    tuner=None,                       # Automatic hyperparameter tuner selection
    overwrite=True,                   # Don't load previous experiments
    seed=42                           # Set a seed to replicate the experiment
)

### Neural Architecture Search
It’s all ready to start exploring the models.

In [7]:
clf.fit(                   # Fit the model
    x=x_train,             # Training features
    y=y_train,             # Training labels
    epochs=None,           # Automatic number of epochs
    callbacks=None,        # No callbacks
    validation_split=0.2,  # Validation data split
    validation_data=None   # Use a portion of training data
)

Trial 3 Complete [00h 01m 31s]
val_loss: 1.2355173826217651

Best val_loss So Far: 0.9400835037231445
Total elapsed time: 00h 04m 27s
INFO:tensorflow:Oracle triggered exit
Epoch 1/7
281/281 [==============================] - 4s 13ms/step - loss: 2.0264 - accuracy: 0.4977
Epoch 2/7
281/281 [==============================] - 4s 13ms/step - loss: 1.3884 - accuracy: 0.6781
Epoch 3/7
281/281 [==============================] - 4s 13ms/step - loss: 1.1449 - accuracy: 0.7263
Epoch 4/7
281/281 [==============================] - 4s 13ms/step - loss: 0.9989 - accuracy: 0.7633
Epoch 5/7
281/281 [==============================] - 4s 13ms/step - loss: 0.8885 - accuracy: 0.7843
Epoch 6/7
281/281 [==============================] - 4s 13ms/step - loss: 0.7974 - accuracy: 0.8043
Epoch 7/7
281/281 [==============================] - 4s 13ms/step - loss: 0.7207 - accuracy: 0.8187


The model found can now be exported, it’s a convolutional multi-layer neural network with the following architecture.

In [8]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 512)               0         
_________________________________________________________________
embedding (Embedding)        (None, 512, 64)           320064    
_________________________________________________________________
dropout (Dropout)            (None, 512, 64)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 508, 256)          82176     
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)              

## Testing the model
The benchmark chosen for this dataset is the experiment reported on the book [Deep Learning with Python by Francois Chollet](https://www.manning.com/books/deep-learning-with-python). The testing crossentropy of the benchmark is ~0.96.

Let’s now test our model on the holdout test set.

In [9]:
# Evaluate the best model with testing data.
current = clf.evaluate(x_test, y_test)

71/71 [==============================] - 0s 6ms/step - loss: 0.9296 - accuracy: 0.7854


In [10]:
previous = 0.9565213431445807
improvement = (abs(current[0] - previous) / previous) * 100.0

print(f'The categorical crossentropy improvement is {round(improvement, 1)}%.')

The categorical crossentropy improvement is 2.8%.


The accuracy of ~0.79 is in line with the previous model. However, we scored a cross entropy of ~0.93 with only three models explored, outperforming the textbook benchmark by almost 3%.

Depending on our hardware and time availability, of course, we could explore even more models for further improvement in the benchmark score.

[**Go back to index >**](https://github.com/MattiaVerticchio/PersonalProjects/blob/master/README_EN.md)